Data obtained from GEO:GSE152058

I have added the three needed files in a folder called 'Files'.


In [ ]:
# Load libraries
library(Seurat)
library(Matrix)
library(writexl)

#Set directory, here you have to change the directory to the ones that contain the files
setwd("C:/Users/ana_m/Downloads/files/")
matrix_dir = "C:/Users/ana_m/Downloads/files/"

#Load the data
barcode.path <- paste0(matrix_dir, "barcodes.tsv.gz")
features.path <- paste0(matrix_dir, "features.tsv.gz")
matrix.path <- paste0(matrix_dir, "matrix.mtx.gz")
mat <- readMM(file = matrix.path)
feature.names = read.delim(features.path,
                           header = TRUE,
                           stringsAsFactors = FALSE)
barcode.names = read.delim(barcode.path,
                           header = TRUE,
                           stringsAsFactors = FALSE)
colnames(mat) = barcode.names$Barcode
rownames(mat) = feature.names$Gene

#Create SeuratObject and add features to the metadata
seurat_obj <- CreateSeuratObject(counts = mat, project = "HDproject", min.cells = 3, min.features = 200)
seurat_obj@meta.data$Condition <- barcode.names$Condition
seurat_obj@meta.data$Celltype <- barcode.names$CellType
seurat_obj@meta.data$Age <- barcode.names$Age
seurat_obj@meta.data$Region <- barcode.names$Region
View(seurat_obj@meta.data)

# Filter and normalize data
seurat_obj <- NormalizeData(seurat_obj)
seurat_obj <- FindVariableFeatures(seurat_obj)
seurat_obj <- ScaleData(seurat_obj)

Idents(object = seurat_obj) <- "Condition"
#Check for variable features info
top10 <- head(VariableFeatures(seurat_obj), 10)
plot1 <- VariableFeaturePlot(seurat_obj)
plot2 <- LabelPoints(plot = plot1, points = top10, repel = TRUE)
plot2

#Percentage of cells in each condition and cell type
celltype <- Idents(seurat_obj) <- seurat_obj$Celltype
condition <- Idents(seurat_obj) <- seurat_obj$Condition
data <- data.frame(celltype, condition)
frequencies <- table(data)
freqdf <- as.data.frame(frequencies)
write_xlsx(freqdf, 'frequencies_newanalysis.xlsx')
View(freqdf)


#Save Seurat Object
saveRDS(seurat_obj, file = "scRNAseq_data.rds")


